In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.metrics import accuracy_score

# Cargar los datasets
df_final_demo_cleanead = pd.read_csv(r'C:\Users\VSPC\Documents\GitHub\New-Web-Study\data\df_final_demo_cleanead.csv')
df_web_data_total = pd.read_csv(r'C:\Users\VSPC\Documents\GitHub\New-Web-Study\data\df_web_data_total.csv')
df_final_experiment_clients_1 = pd.read_csv(r'C:\Users\VSPC\Documents\GitHub\New-Web-Study\data\raw\df_final_experiment_clients.txt')

# Merging the demo and web data on client_id
merged_data = pd.merge(df_final_demo_cleanead, df_web_data_total, on='client_id', how='inner')

# Merging the experiment clients data to add the target variable 'Variation'
final_data = pd.merge(merged_data, df_final_experiment_clients_1, on='client_id', how='left')

# Dropping rows where 'Variation' is NaN
final_data_clean = final_data.dropna(subset=['Variation'])

label_encoder = LabelEncoder()
final_data_clean['Variation'] = label_encoder.fit_transform(final_data_clean['Variation'])

# Definir las características (X) y el objetivo (y)
X = final_data_clean.drop(columns=['Variation', 'client_id', 'visitor_id', 'visit_id', 'date_time', 'date', 'time'])
y = final_data_clean['Variation']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo LGBMClassifier
lgb_model = lgb.LGBMClassifier(
    objective='binary',
    is_unbalance=True,
    num_leaves=31,
    learning_rate=0.05,
    feature_fraction=0.9,
    n_estimators=100
)

# Entrenar el modelo con early stopping
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='logloss',
    early_stopping_rounds=10
)

# Hacer predicciones
y_pred_lgb = lgb_model.predict(X_test)

# Calcular la precisión
accuracy_lgb = accuracy_score(y_test, y_pred_lgb)

print(f"Precisión del modelo LightGBM: {accuracy_lgb}")


C:\Users\VSPC\AppData\Local\Temp\ipykernel_116\3465439301.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data_clean['Variation'] = label_encoder.fit_transform(final_data_clean['Variation'])


TypeError: LGBMClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'